In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 4

def train_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    out = [numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = True, collate_fn=test_collate, num_workers=0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    def __init__(self,
                 input_size = 2,
                 embedding_size = 8,
                 hidden_size = 16):
        
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear(x))
        hidden = self.lstm(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    def __init__(self, embedding_size=8, hidden_size=16, output_size=2):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [5]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(train_loader):
        
        inp, out = sample_batch
        # preprocessing more ????
        inp = inp[:,0,:,:]
        out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #outoput whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        loss = 0
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += torch.sqrt(criterion(decoder_output[:, :2], target[:, di, :2]))

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
#         output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        iterator.set_postfix(loss=(loss.item()*_input.size(0) / (counter * train_loader.batch_size)))

In [6]:
device = "cuda"
encoder = EncoderRNN(input_size=2)
decoder = DecoderRNN(output_size=2)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 7

for epoch in range(1, num_epoch + 1):
        train(encoder, decoder, device, val_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

In [7]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
        _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 2)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(4):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [8]:
output = infer_absolute(test_loader, encoder, decoder)

In [9]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
23642,"[2077.922119140625, 641.70068359375]","[2076.721923828125, 640.9498901367188]","[2075.847900390625, 640.1170654296875]","[2074.945068359375, 639.2413330078125]","[2073.97265625, 638.3536376953125]","[2072.976318359375, 637.4682006835938]","[2071.9619140625, 636.591552734375]","[2070.931396484375, 635.7241821289062]","[2069.889892578125, 634.8663330078125]","[2068.84375, 634.0194702148438]",...,"[2058.450927734375, 625.942626953125]","[2057.59912109375, 625.2306518554688]","[2056.743896484375, 624.4974975585938]","[2055.8818359375, 623.7520141601562]","[2055.010498046875, 623.0096435546875]","[2054.130126953125, 622.2879028320312]","[2053.241455078125, 621.602294921875]","[2052.34619140625, 620.9638671875]","[2051.448974609375, 620.3782958984375]","[2050.5556640625, 619.8462524414062]"
24735,"[1748.373291015625, 448.5286865234375]","[1747.3543701171875, 448.58380126953125]","[1747.4132080078125, 448.9993896484375]","[1747.2149658203125, 449.1559143066406]","[1747.0316162109375, 449.4153747558594]","[1746.8189697265625, 449.6625671386719]","[1746.5869140625, 449.9256591796875]","[1746.331298828125, 450.1965026855469]","[1746.0538330078125, 450.4757080078125]","[1745.75439453125, 450.7618713378906]",...,"[1741.2562255859375, 454.27703857421875]","[1740.78369140625, 454.62884521484375]","[1740.311767578125, 454.98492431640625]","[1739.843505859375, 455.344482421875]","[1739.38134765625, 455.7065124511719]","[1738.9278564453125, 456.0699462890625]","[1738.4852294921875, 456.43359375]","[1738.0556640625, 456.79638671875]","[1737.640869140625, 457.1571960449219]","[1737.2423095703125, 457.51495361328125]"
32038,"[1805.400390625, 423.6491394042969]","[1803.9720458984375, 422.90338134765625]","[1803.0587158203125, 421.9989929199219]","[1802.145263671875, 421.0779724121094]","[1801.1397705078125, 420.1469421386719]","[1800.1005859375, 419.21392822265625]","[1799.0341796875, 418.2853088378906]","[1797.9420166015625, 417.35931396484375]","[1796.8299560546875, 416.4344787597656]","[1795.705810546875, 415.5116271972656]",...,"[1784.471923828125, 406.6732177734375]","[1783.557861328125, 405.91046142578125]","[1782.633544921875, 405.10504150390625]","[1781.6929931640625, 404.2681884765625]","[1780.7335205078125, 403.4273986816406]","[1779.7554931640625, 402.61541748046875]","[1778.7620849609375, 401.85882568359375]","[1777.75927734375, 401.1721496582031]","[1776.7567138671875, 400.5579833984375]","[1775.768310546875, 400.009521484375]"
12929,"[1764.9486083984375, 390.24554443359375]","[1763.9102783203125, 389.46942138671875]","[1763.0963134765625, 388.7174377441406]","[1762.27685546875, 387.900390625]","[1761.397705078125, 387.07666015625]","[1760.50048828125, 386.2558898925781]","[1759.5909423828125, 385.44384765625]","[1758.669921875, 384.6407470703125]","[1757.740234375, 383.8464660644531]","[1756.8056640625, 383.06170654296875]",...,"[1747.2921142578125, 375.47119140625]","[1746.5086669921875, 374.81182861328125]","[1745.725341796875, 374.1312561035156]","[1744.9383544921875, 373.431396484375]","[1744.1451416015625, 372.7206115722656]","[1743.34423828125, 372.01190185546875]","[1742.5352783203125, 371.3197326660156]","[1741.71875, 370.6571044921875]","[1740.89599609375, 370.03375244140625]","[1740.06982421875, 369.455322265625]"
19110,"[2087.665771484375, 652.527099609375]","[2086.396240234375, 651.7200927734375]","[2085.48779296875, 650.85888671875]","[2084.5830078125, 649.955322265625]","[2083.603759765625, 649.0451049804688]","[2082.598876953125, 648.1383666992188]","[2081.57568359375, 647.2410278320312]","[2080.53564453125, 646.3524780273438]","[2079.483642578125, 645.471923828125]","[2078.426025390625, 644.6002807617188]",...,"[2067.909912109375, 636.3643798828125]","[2067.057373046875, 635.6669921875]","[2066.20166015625, 634.9420166015625]","[2065.33740234375, 634.1939086914062]","[2064.46240234375, 633.437255859375]","[2063.574951171875, 632.6923217773438]","[2062.67651

In [10]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)

In [11]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
23642,2077.922119,641.700684,2076.721924,640.949890,2075.847900,640.117065,2074.945068,639.241333,2073.972656,638.353638,...,2054.130127,622.287903,2053.241455,621.602295,2052.346191,620.963867,2051.448975,620.378296,2050.555664,619.846252
24735,1748.373291,448.528687,1747.354370,448.583801,1747.413208,448.999390,1747.214966,449.155914,1747.031616,449.415375,...,1738.927856,456.069946,1738.485229,456.433594,1738.055664,456.796387,1737.640869,457.157196,1737.242310,457.514954
32038,1805.400391,423.649139,1803.972046,422.903381,1803.058716,421.998993,1802.145264,421.077972,1801.139771,420.146942,...,1779.755493,402.615417,1778.762085,401.858826,1777.759277,401.172150,1776.756714,400.557983,1775.768311,400.009521
12929,1764.948608,390.245544,1763.910278,389.469421,1763.096313,388.717438,1762.276855,387.900391,1761.397705,387.076660,...,1743.344238,372.011902,1742.535278,371.319733,1741.718750,370.657104,1740.895996,370.033752,1740.069824,369.455322
19110,2087.665771,652.527100,2086.396240,651.720093,2085.487793,650.858887,2084.583008,649.955322,2083.603760,649.045105,...,2063.574951,632.692322,2062.676514,631.979126,2061.769287,631.312744,2060.858154,630.701355,2059.949951,630.146545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28955,750.170288,1388.886108,749.367798,1388.631958,748.941589,1388.897583,748.564270,1388.897827,748.177734,1388.996582,...,738.946228,1392.152222,738.515442,1392.391602,738.090637,1392.638916,737.672485,1392.893921,737.261963,1393.156372
28797,1741.100830,467.703796,1740.518066,466.935944,1739.912231,466.597046,1739.357422,466.132935,1738.747437,465.698792,...,1725.082520,454.948151,1724.426758,454.420013,1723.776855,453.908661,1723.134155,453.414764,1722.500122,452.938507
10242,571.922485,1286.194824,571.214539,1285.260010,570.762756,1285.109619,570.314941,1284.545654,569.792114,1284.165039,...,556.251221,1275.082764,555.541260,1274.636475,554.834656,1274.205933,554.133423,1273.792603,553.439514,1273.396851


In [12]:
df2.to_csv("outputs7ep.csv", index=True)